In [1]:
import pandas as pd
import numpy as np
import time
import ast
import datetime
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFECV
import numpy as np
import seaborn as sns
from sklearn.model_selection import cross_val_score
from sklearn import metrics
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.manifold import TSNE
warnings.filterwarnings("ignore")
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import anderson
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import preprocessing

# About the Dataset

In [22]:

#nextcell
ratings = pd.read_csv('/Users/ankitkothari/Documents/gdrivre/UMD/MSML-602-DS/final_project/ratings_small.csv')
movies = pd.read_csv('/Users/ankitkothari/Documents/gdrivre/UMD/MSML-602-DS/final_project/movies_metadata_features.csv')



# Data Cleaning

## Dropping Columns

In [23]:
movies.drop(columns=['Unnamed: 0'],inplace=True)
ratings = pd.merge(movies,ratings).drop(['genres','timestamp','imdb_id','overview','popularity','production_companies','production_countries','release_date','revenue','runtime','vote_average','year','vote_count','original_language'],axis=1)


In [92]:
usri = int(input()) #587 #15 #468
select_user = ratings.loc[ratings['userId'] == usri]



15


## Finding Similarity Matrix
### Creating a Pivot Table of Title against userId for ratings

In [94]:
userRatings = ratings.pivot_table(index=['title'],columns=['userId'],values='rating')
userRatings = userRatings.dropna(thresh=10, axis=1).fillna(0,axis=1)



In [95]:
corrMatrix = userRatings.corr(method='pearson')
#corrMatrix = userRatings.corr(method='spearman')
#corrMatrix = userRatings.corr(method='kendall')


### Creating Similarity Matrix using Pearson Correlation method

In [96]:
def get_similar(usrid):
    similar_ratings = corrMatrix[usrid]
    similar_ratings = similar_ratings.sort_values(ascending=False)
    return similar_ratings



# Recommendation

In [97]:
moidofotus = [0,0,0,0]
s_m = pd.DataFrame()
s_m = s_m.append(get_similar(usri), ignore_index=True)
for c in range(0,4):
    moidofotus[c]=s_m.columns[c]

if moidofotus[0] == usri:
    moidofotus.pop(0)
print(moidofotus)

movie_match=[]
for i in moidofotus:
    select_user = ratings.loc[ratings['userId'] == i]
    #print(select_user)
    print("For user", i)
    final_use = select_user.loc[select_user['rating'] >= 4.0].sort_values(by=['rating'],ascending=False).iloc[0:10,:]
    print(final_use['title'])
    movie_match.append(final_use['title'].to_list())

[388, 461, 509]
For user 388
14337    The Chronicles of Riddick: Dark Fury
6683                         Mr. & Mrs. Smith
4016                            Love Actually
11089                      The Golden Compass
9419                       Notes on a Scandal
4792                         Dawn of the Dead
5204                                 Zatoichi
10750                                  Postal
10478                     Across the Universe
7090                      Cockles and Muscles
Name: title, dtype: object
For user 461
818                               The Million Dollar Hotel
2030                                       Monsoon Wedding
14347                 The Chronicles of Riddick: Dark Fury
1126                                        Under the Sand
5399                                          Two Brothers
13730                                    Windows on Monday
13107    Shriek If You Know What I Did Last Friday the ...
12576                                   The Garden of Eden

In [98]:
select_user['title']

3                                    Scream 3
72                             Reindeer Games
207        The Flintstones in Viva Rock Vegas
272                         The Perfect Storm
327                        The Way of the Gun
                         ...                 
14106                 Just a Question of Love
14195                          Totally Blonde
14355    The Chronicles of Riddick: Dark Fury
14393                          Manitou's Shoe
14416                                    Beat
Name: title, Length: 149, dtype: object

## Performance Evaluation

In [99]:
movies_suggested_and_he_watched=0
total_suggest_movies = 0
for movies in movie_match:
    total_suggest_movies=total_suggest_movies+len(movies)
    for movie in movies:
        if movie in select_user['title'].to_list():
            movies_suggested_and_he_watched=movies_suggested_and_he_watched+1
print(movies_suggested_and_he_watched)
print(total_suggest_movies)

27
30
